In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import kraken as kr
import matplotlib.pyplot as plt
import numpy as np
import scanpy as sc

In [ ]:
filenames = ['/home/sebastian/data/placenta_rebuttal/156KS_files/156KS_Decoded_ImmunePanel.csv',
'/home/sebastian/data/placenta_rebuttal/156KS_files/156KS_Decoded_KISS and CGA.csv',
'/home/sebastian/data/placenta_rebuttal/156KS_files/156KS_Decoded_PlacentaPanel.csv']

coord_tables=[]

for file_coordinates in filenames:
    coord_tables.append(pd.read_csv(file_coordinates))
    
adata = sc.read_h5ad('/home/sebastian/data/Placenta_normalized_Seurat_markers.h5ad')

coordinates=pd.concat(coord_tables)

In [ ]:
coordinates.gene.replace('HBA1/2','HBA1', inplace=True)
coordinates.gene.replace('KISS','KISS1', inplace=True)

coordinates.index = np.arange(coordinates.shape[0])
coordinates

In [ ]:

bg = kr.PixelMap(np.load('bg.npy').T,upscale=0.2)

sdata = kr.SpatialData(coordinates.gene, 
                         coordinates.X, 
                         coordinates.Y,
                        pixel_maps=[bg],scanpy=adata,synchronize=True)

# sdata = sdata[sdata.gene_id.isin(sdata.stats[sdata.counts>50].gene_ids)].spatial[:4000,:4000]
# bg = kr.KDEProjection(sdata,bandwidth=12, upscale=0.1)
# sdata.pixel_maps.append(kr.PixelMap(bg.data.sum(-1),upscale=0.1))

In [ ]:
plt.figure(figsize=(12,12))

sdata.spatial[:2000,:2000].scatter(alpha=1,marker='x')

In [ ]:
plt.figure(figsize=(12,12))

sdata[sdata.g.isin(sdata.genes[sdata.stats.counts>1000])].plot_overview()

In [ ]:
plt.figure(figsize=(15,15))
sdata[sdata.g.isin(sdata.genes[sdata.stats.counts>1000])].graph.plot_entropy()

In [ ]:
plt.figure(figsize=(15,15))

sdata.scanpy.compare_counts()

In [ ]:
sdata_subset = sdata[sdata.g.isin(('PLAC4','FLT1',))]
sdata_subset

In [ ]:
sdata.uns

In [ ]:
import squidpy

squidpy.gr.co_occurrence(sdata_subset,cluster_key='gene')


In [ ]:
squidpy.pl.co_occurrence(sdata_subset,cluster_key='gene')

In [ ]:
plt.figure(figsize=(15,15))
sdata.scatter(marker='x',alpha=1,color='grey')
sdata[sdata.gene_annotations=='FN1'].scatter(marker='x',alpha=1,color='lime')
sdata[sdata.gene_annotations=='HBA1'].scatter(marker='x',alpha=1,color='r')


In [ ]:
sdata.plot_radial_distribution(n_neighbors=10, bins=np.linspace(0,90,90))

In [ ]:
sdata[sdata.gene_annotations.isin(sdata.stats[sdata.stats.counts>50].index)]

In [ ]:
assert False

In [ ]:
import scipy.spatial
import libpysal as ps
from pointpats import PointPattern, PoissonPointProcess, as_window, g#, F, J, K, L, Genv, Fenv, Jenv, Kenv, Lenv
from pointpats.distance_statistics import G,F,J,K,L

In [ ]:
pp = PointPattern(np.array(sdata[sdata.gene_annotations=='KISS1'][['X','Y']]))

In [ ]:
gp1 = G(pp, intervals=50)
gp1.plot(qq=True)

In [ ]:
kp1 = K(pp)
kp1.plot()

In [ ]:
lp1 = L(pp)
lp1.plot()

In [ ]:
assert False

In [ ]:
sdata = sdata.spatial[:1000,:1000]
#kr.SpatialData(coordinates.gene[subset_mask], coordinates.X[subset_mask], coordinates.Y[subset_mask])#)gene_tag='gene')


In [ ]:
np.array(sdata.coordinates).T

In [ ]:
sdata.X

hex_to_cartesian = np.array([[0.5,0],[3**0.5/2,3**0.5]])

cartesian_to_hex = np.linalg.inv(hex_to_cartesian)
# hex_coords = np.inner(np.array(sdata.coordinates).T,cartesian_to_hex[:,:2])


plt.figure(figsize=(7,7))
# plt.scatter()

plt.scatter(*hex_coords.T)
# plt.scatter(*sdata.coordinates)



In [ ]:
# H = np.array(-count_matrix.multiply(count_matrix_log.todense()).sum(1)).flatten()


In [ ]:
Dkl = sdata.knn_clean(n_neighbors=20)

In [ ]:
# plt.figure(figsize=(14,14))

# plt.scatter(spatial.X,spatial.Y,c=spatial.gene_ids,cmap='nipy_spectral',s=np.array(-H_.sum(1)).flatten()**2)#c=H)
# plt.colorbar()

In [ ]:
plt.figure(figsize=(14,14))

mask = Dkl<(Dkl.mean()*3.5)
plt.scatter(sdata.X[mask],sdata.Y[mask],c=sdata.gene_ids[mask],cmap='nipy_spectral',
            s=10)
plt.colorbar()

In [ ]:
sdata[sdata.gene_annotations=='PLAC4'].scatter(color='r')

In [ ]:
plt.figure(figsize=(10,10))
out = sdata.spatial_decomposition(('Cpne5',),('Cpne5',),n_neighbors=15)

In [ ]:


# plt.imshow((out//out.sum(1)).sum(-1))
# cum = out.sum(1).sum(1)
# plt.imshow(out.sum(-1))#.T/cum)

In [ ]:
pptt,pptr,clrs=out

In [ ]:
hists.shape

In [ ]:
# kewl,binsx,binsy = (np.histogram2d(pptt,pptr, bins=20))

from skimage.filters import gaussian

# bg = gaussian(bg,sigma=15)

hists = np.array([gaussian(np.histogram2d(pptt[clrs==c],pptr[clrs==c],bins=(binsx,binsy))[0],sigma=2) for c in range(len(sdata.gene_classes))]).T
hists_norm = ((hists/hists.sum(0)[:,None]))

In [ ]:
# plt.imshow(hists.sum(-1))
out=hists_norm.reshape((-1,hists.shape[-1]))
out.std(1)

In [ ]:
# plt.figure(figsize=(10,10))
plt.imshow(out,aspect='auto')

In [ ]:
from sklearn.decomposition import NMF as NMF

nmf = NMF(n_components=4, alpha_H=5.0, alpha_W=0.0,  l1_ratio=0.0)
facs = nmf.fit_transform(out)
comps = nmf.components_
facs.shape,comps.shape

In [ ]:
facs.shape,nmf.components_.shape

for comp in range(comps.shape[0]):
    plt.figure(figsize=(10,7))
    
    print(sdata.gene_classes[comps[comp].argsort()])

    plt.subplot(121)
    plt.bar(np.arange(comps.shape[1]),comps[comp])#comps[comp])

    plt.subplot(122)
    img  = facs[:,comp].reshape(hists.shape[:2])
    plt.imshow(img.T**1.1/img.sum(1),aspect='auto')

In [ ]:
# for i in range(hists.shape[-1]):
#     plt.figure(figsize=(7,7))
#     plt.imshow(hists[:,:,i].T**1.2/hists[:,:,i].sum(1))
#     plt.title(sdata.gene_classes[i])

In [ ]:
coordinates_cortex = pd.read_csv('/home/sebastian/data/ssamLiteDataFinal/CodeluppiosmFISH/coordinates.csv')

# mask = (coordinates_cortex.X>1200)&(coordinates_cortex.Y<1000)
sdata = kr.SpatialData(coordinates_cortex.genes, coordinates_cortex.X, coordinates_cortex.Y)#)gene_tag='gene')

# adata_cortex=sc.read_h5ad("data/adata_processed_sc.h5ad")

In [ ]:
spatial_cortex.plot_entropy()

In [ ]:
Dkl = spatial_cortex.knn_clean(n_neighbors=20)

In [ ]:
Dkl

In [ ]:
plt.figure(figsize=(15,7))

plt.subplot(121)
plt.scatter(spatial_cortex.X,spatial_cortex.Y,c=spatial_cortex.gene_ids,s=1/(0.01+Dkl**2), cmap='nipy_spectral')

# plt.xlim((1500,1700))
# plt.ylim((300,500))


plt.subplot(122)
plt.scatter(spatial_cortex.X,spatial_cortex.Y,c=spatial_cortex.gene_ids,s=(0.01+Dkl**3), cmap='nipy_spectral')

plt.xlim((1500,1700))
plt.ylim((300,500))

In [ ]:
out = spatial_cortex.spatial_decomposition(('Acta2',))

In [ ]:
plt.figure(figsize=(15,25))
spatial.scatter('KISS1',marker='.')

In [ ]:
mus = np.array([14,17,20,40,42,80])
sigmas=[1,2]
thresholds=[0.2,0.15]

def normal_pdf(mu,sigma):
    return( 1/(sigma**2*2*np.pi)**0.5 * np.exp(-(mu**2)/(2*sigma**2))  )

x = np.linspace(0,100,500)

for i in range(len(thresholds)):
    threshold=thresholds[i]
    sigma=sigmas[i]

    y = np.zeros_like(x)

    for mu in mus:
        y+=normal_pdf(x-mu,sigma)
        
    plt.figure()

    plt.vlines([mus],0,0.3,colors='k', alpha=0.2)  
    plt.hlines(threshold,x.min(),x.max(),colors='k')  

    filtered = np.ones_like(x)*threshold
    filtered[y<=threshold]=np.nan

    plt.plot(x,y)

    plt.plot(x,filtered,c='r',linewidth=6)

    plt.title(f'sigma:{sigma} - threshold:{threshold}')

In [ ]:
mus = np.array([[14,20,40,42,79],
                [24,33,60,80,81]]).T
# sigma=4
thresholds=[0.09,0.06,0]

clrs = ['k','royalblue','red']

def normal_pdf(mu,sigma):
    return( 1/(sigma**2*2*np.pi)**0.5 * np.exp(-(mu**2)/(2*sigma**2))  )

x = np.linspace(0,100,500)
x=np.vstack((x,x))
for i in range(len(thresholds)):
    threshold=thresholds[i]

    y = np.zeros_like(x)

    for mu in mus:
        y+=normal_pdf(x-mu[:,None],sigma)
        
    plt.figure()

    plt.vlines([mus],0,0.3,colors='k', alpha=0.1)  
    plt.hlines(threshold,x.min(),x.max(),colors='k')  

    c = np.array([y_.argmax()+1 if y_.max()>threshold else 0 for y_ in y.T])

    for j in range(len(clrs)):
        filtered = np.ones_like(x[0])*threshold

        filtered[c!=j]=np.nan
        if j==0: lw=3 
        else:lw=7
        plt.plot(x[0],filtered,c=clrs[j],linewidth=lw)

    for j in range(x.shape[0]):
        plt.plot(x[j],y[j],c=clrs[j+1], alpha=0.3)
    plt.title(f'sigma:{sigma} - threshold:{threshold}')
    

    

In [ ]:
# coordinates_cortex.loc[coordinates_cortex.genes=='Lamp5','X']+=np.random.normal(size=((coordinates_cortex.genes=='Lamp5').sum(),))*20
# coordinates_cortex.loc[coordinates_cortex.genes=='Lamp5','Y']+=np.random.normal(size=((coordinates_cortex.genes=='Lamp5').sum(),))*20
# spatial_cortex = kr.SpatialData(coordinates_cortex.genes, coordinates_cortex.X, coordinates_cortex.Y)#)gene_tag='gene')


In [ ]:
coordinates.loc[coordinates.gene=='KISS1','X']+=np.random.normal(size=((coordinates.gene=='KISS1').sum(),))*20
coordinates.loc[coordinates.gene=='KISS1','Y']+=np.random.normal(size=((coordinates.gene=='KISS1').sum(),))*20
spatial = kr.SpatialData(coordinates.gene, coordinates.X, coordinates.Y)#)gene_tag='gene')


In [ ]:
plt.figure(figsize=(15,25))
spatial_cortex.scatter('Lamp5',marker='.')

In [ ]:
# spatial_cortex = kr.SpatialData(coordinates_cortex.genes, coordinates_cortex.X, coordinates_cortex.Y)#)gene_tag='gene')
spatial.plot_entropy()

In [ ]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=12)

coords = np.stack([spatial.data.X, spatial.data.Y]).T
knn.fit(coords)
distances, indices = knn.kneighbors(coords)

knn_cells = np.zeros_like(indices)
for i in range(indices.shape[1]):
    knn_cells[:, i] = spatial.data['gene_id'].iloc[indices[:, i]]


In [ ]:
plt.figure(figsize=(8,25))

H = []

for i in range(spatial.data['gene_id'].max()+1):
    x = knn_cells[spatial.data.gene_id==i]

    _, n_x = np.unique(x[:,1:], return_counts=True)
        
    p_x = n_x/(n_x.sum())
    print(spatial.gene_classes[i], p_x.max())
    
    h_i = -(p_x*np.log2(p_x)).sum()
    H.append(h_i)
    plt.plot((p_x)+i/2)
    plt.plot(np.zeros_like(p_x)+i/2,c='grey')
    
# plt.bar(np.arange(len(H)),H)    
# plt.yticks(np.arange(len(H))/2,spatial_cortex.gene_classes, rotation=90)
plt.yticks(np.arange(len(H))/2,spatial.gene_classes, rotation=0)

plt.show()
# plt.yscale('log')

In [ ]:
list({"a":1,"c":1,"b":1})

In [ ]:
spatial_cortex.plot_overview()

In [ ]:
(adata_cortex,spatial_cortex) = kr.synchronize(adata_cortex,spatial_cortex,0,0)

In [ ]:
plt.figure(figsize=(10,10))
kr.compare_counts(adata_cortex,spatial_cortex)

In [ ]:
fig = plt.figure(figsize=(7,7))

ax = fig.subplots(1)

ax.plot(np.random.normal(size=(30,)))
# plt.xticks(np.arange(30),np.arange(30))

print("Dot per inch(DPI) for the figure is: ", fig.dpi)
bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
width, height = bbox.width, bbox.height
print("Axis sizes are(in pixels):", width, height)


In [ ]:
import scvi


adata_spatial = scvi.data.smfish(save_path='data/')

In [ ]:
adata_spatial_anterior = scpy.datasets.visium_sge(
    sample_id="V1_Mouse_Brain_Sagittal_Anterior"
)

In [ ]:
scpy.pl.spatial(adata_spatial_anterior)

In [ ]:
adata_spatial_anterior.uns['spatial'].keys()

In [ ]:
import scipy
import sys

In [ ]:
                      
X = scipy.sparse.csr_matrix((np.ones((coordinates.shape[0],)),
                             (np.arange(coordinates.shape[0]),
                                          coordinates.gene.apply(lambda x: newobj.var.index.get_loc(x)))),
                           shape=(coordinates.shape[0], len(adata.var)))
                              
# for i,c in coordinates.iterrows():
#     newobj.X[i,newobj.var.index==c.gene]=1
newobj = scpy.AnnData(X, var=adata.var)
    
newobj.obsm['spatial']  = np.array(coordinates[['X','Y']])
# newobj.obsm['Y']  = np.array(coordinates['Y'])

In [ ]:
                          
scpy.pl.spatial(newobj, spot_size=0.5)

In [ ]:
# newobj.obsm['spatial']
adata_spatial_anterior.uns['spatial']

In [ ]:
import squidpy as sq
adata_squid= sq.datasets.slideseqv2()

In [ ]:
scpy.pl.spatial(adata_squid, color="cluster", spot_size=10)

In [ ]:
adata_squid.shape

In [ ]:
plt.imshow(np.linspace)